In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, trim, upper, lower, initcap, current_timestamp, 
    lit, coalesce, when, regexp_replace, length, row_number, to_timestamp,count,sum
)

from pyspark.sql.types import IntegerType, StringType, TimestampType, StructType, StructField
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
from pyspark.sql.functions import regexp_replace, col, current_timestamp
from pyspark.sql import DataFrame


In [0]:
catalog_name = "atendimento_catalog"
bronze_db_name = "bronze"
silver_db_name = "silver"

In [0]:
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {silver_db_name}")

In [0]:
df_bronze_ft_custos = spark.table("bronze.ft_custos")
display(df_bronze_ft_custos)


---

## 🔍 Verificação de Inconsistências nas Colunas

Nesta etapa realizamos a checagem de valores inconsistentes da tabela **ft_custos** antes de gerar a camada *Silver*. As validações incluem:

### **1. Formato incorreto nas colunas**

As colunas analisadas foram:

* **id_custo** → deve conter apenas números inteiros (ex: `1234`)
* **id_chamado** → deve conter apenas números inteiros (ex: `9876`)
* **custo** → deve conter apenas números ou números com vírgula representando decimais (ex: `12`, `12,50`, `0,0042`)

Valores considerados **incorretos** incluem:

* letras ou símbolos
* espaços
* valores em formato textual
* valores com ponto em vez de vírgula para decimais
* campos vazios ou não numéricos

---

### **2. Identificação de duplicatas**

Também verificamos se há IDs repetidos:

* **id_custo** duplicado
* **id_chamado** duplicado

Esses casos indicam registros possivelmente inconsistentes ou replicados na ingestão.

---


---
## 🔍 Verificação de Valores Nulos

Antes de avançar para a camada *Silver*, verificamos se alguma coluna da tabela contém valores nulos. Isso ajuda a identificar dados incompletos ou possíveis falhas na ingestão.

O código soma a quantidade de `NULL` em cada coluna e exibe o resultado, permitindo saber rapidamente onde existe inconsistência e se será necessário tratar, corrigir ou remover registros.

---


In [0]:
df_bronze_ft_custos.select([
    sum(col(c).isNull().cast("int")).alias(f"{c}_nulls")
    for c in df_bronze_ft_custos.columns
]).display()

In [0]:


df_invalid = df_bronze_ft_custos.select(

    # inválidos em id_custo
    sum(
        (~col("id_custo").cast("string").rlike("^[0-9]+$")).cast("int")
    ).alias("id_custo_invalid"),

    # inválidos em id_chamado
    sum(
        (~col("id_chamado").cast("string").rlike("^[0-9]+$")).cast("int")
    ).alias("id_chamado_invalid"),

    # inválidos em custo (decimal com vírgula)
    sum(
        (~col("custo").cast("string").rlike("^[0-9]+(,[0-9]+)?$")).cast("int")
    ).alias("custo_invalid")
)

df_invalid.display()


In [0]:
df_bronze_ft_custos_invalid_rows = df_bronze_ft_custos.filter(
    ~col("id_custo").cast("string").rlike("^[0-9]+$") |
    ~col("id_chamado").cast("string").rlike("^[0-9]+$") |
    ~col("custo").cast("string").rlike("^[0-9]+(,[0-9]+)?$")
)
df_bronze_ft_custos_invalid_rows.display()


In [0]:
#verificação de id_custo duplicados
df_ft_custos_id_custos_dups_count = (
    df_bronze_ft_custos
        .groupBy("id_custo")
        .agg(count("*").alias("qtd"))
        .filter(col("qtd") > 1)
)
df_ft_custos_id_custos_dups_count.display()

In [0]:
#verificação de id_chamados duplicados
df_ft_id_chamados_dups_count = (
    df_bronze_ft_custos
        .groupBy("id_chamado")
        .agg(count("*").alias("qtd"))
        .filter(col("qtd") > 1)
)
df_ft_id_chamados_dups_count .display()

---

## 🛠️ Transformação Silver — Função `transform_ft_custos`

A função **`transform_ft_custos`** aplica as regras de tratamento necessárias para gerar a camada *Silver* da tabela **ft_custos**, garantindo padronização de tipos, limpeza de dados e criação do timestamp de processamento.

### ✅ Regras aplicadas

**1. Preservação do `ingestion_timestamp`**
O timestamp original da Bronze é mantido sem alterações.

**2. Conversão e padronização dos tipos**

* **id_chamado → IntegerType**
  Convertido para inteiro, garantindo que seja tratado como identificador numérico.
* **id_custo → IntegerType**
* **custo → Decimal(18,8)**
  Antes da conversão, o valor passa por uma limpeza para garantir o formato numérico válido.

### ✅ Tratamento da coluna *custo*

A coluna pode vir com símbolos ou com vírgula como separador decimal. Por isso:

1. Remove caracteres inválidos (`[^0-9,.-]`), mantendo apenas dígitos e pontuações numéricas.
2. Substitui vírgulas por pontos, pois o Spark só reconhece ponto como decimal.
3. Finalmente, converte para `decimal(18,8)`, garantindo precisão adequada.

### ✅ Filtragem de valores nulos (NOT NULL)

Somente registros onde:

* `id_chamado`
* `id_custo`
* `custo`

não são nulos permanecem na Silver(apesar de não existirem).

### ✅ Adição do `processed_timestamp`

Um timestamp é criado no momento da transformação para indicar quando o registro entrou na Silver.

---

In [0]:
def transform_ft_custos(df_bronze: DataFrame) -> DataFrame:
    """
    Tratamento Silver da tabela ft_custos.

    Regras:
    - Mantém ingestion_timestamp da Bronze
    - id_chamado -> IntegerType (NOT NULL)
    - id_custo -> StringType (NOT NULL)
    - custo -> Decimal(18,8) (NOT NULL)
    - Adiciona processed_timestamp
    """

    df_silver = (
        df_bronze
            # Conversão de tipos
            .withColumn("id_chamado", col("id_chamado").cast("int"))
            .withColumn("id_custo", col("id_custo").cast("int"))

            # Tratamento da coluna custo (string → decimal)
            .withColumn("custo", regexp_replace(col("custo"), "[^0-9,.-]", ""))
            .withColumn("custo", regexp_replace(col("custo"), ",", "."))
            .withColumn("custo", col("custo").cast("decimal(18,8)"))

            # NOT NULL enforcement
            .filter(
                col("id_chamado").isNotNull() &
                col("id_custo").isNotNull() &
                col("custo").isNotNull()
            )

            # Timestamp do processamento Silver
            .withColumn("processed_timestamp", current_timestamp())
    )

    return df_silver


In [0]:
df_silver_ft_custos = transform_ft_custos(df_bronze_ft_custos)

In [0]:
display(df_silver_ft_custos )

In [0]:
#verificação do tratamento 
df_silver_ft_custos_invalid_rows = df_silver_ft_custos.filter(
    ~col("id_custo").cast("string").rlike("^[0-9]+$") |
    ~col("id_chamado").cast("string").rlike("^[0-9]+$") |
    ~col("custo").cast("string").rlike("^[0-9]+(.[0-9]+)?$")
)

df_silver_ft_custos_invalid_rows.display()

In [0]:
df_silver_ft_custos.select([
    sum(col(c).isNull().cast("int")).alias(f"{c}_nulls")
    for c in df_silver_ft_custos.columns
]).display()

In [0]:
# Escreve a tabela ft_custos na camada Silver.
#
# Parâmetros:
# - df_silver: DataFrame já transformado.
# - mode: Modo de escrita (default: overwrite). Pode ser:
#     - 'overwrite'
#     - 'append'
#     - 'error'
#     - 'ignore'
#
# Caminho Final:
# atendimento_catalog.silver.ft_custos  

def write_ft_custos_silver(df_silver: DataFrame, mode: str = "overwrite") -> None:
    (
        df_silver
        .write
        .format("delta")
        .mode(mode)
        .option("overwriteSchema", "true")
        .saveAsTable("atendimento_catalog.silver.ft_custos")
    )
    
    print("Tabela Silver ft_custos escrita com sucesso!")


In [0]:
write_ft_custos_silver(df_silver_ft_custos)